In [ ]:
import gdown
import zipfile

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Screws_2024_07_15')


In [ ]:
!pip install opencv-python


In [ ]:
!pip install torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import cv2
import numpy as np
import os

# Path to your image directory and output directory
image_directory = '/content/drive/MyDrive/Screws_2024_07_15'
output_directory = '/content/drive/MyDrive/CountingChallenge/Non_AI/output'
os.makedirs(output_directory, exist_ok=True)

def process_image(image_path):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f'Failed to load image: {image_path}')
        return None, None, 0

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Image preprocessing
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)  # Adjusted blur kernel size

    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Morphological operations
    kernel = np.ones((3, 3), np.uint8)  # Adjusted kernel size
    morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    morphed = cv2.morphologyEx(morphed, cv2.MORPH_OPEN, kernel, iterations=1)

    # Find contours
    contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Debugging: Check number of contours found
    print(f'Number of contours detected: {len(contours)}')

    # Filter contours based on area
    min_contour_area = 100  # Adjusted minimum contour area
    max_contour_area = 5000  # Adjusted maximum contour area
    filtered_contours = [c for c in contours if min_contour_area < cv2.contourArea(c) < max_contour_area]

    # Debugging: Check filtered contours
    print(f'Number of filtered contours: {len(filtered_contours)}')

    # Create a mask image
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, filtered_contours, -1, (255), thickness=cv2.FILLED)

    # Create an output image with masks applied
    output = cv2.bitwise_and(image, image, mask=mask)

    # Count the number of contours
    num_items = len(filtered_contours)

    return output, mask, num_items

# Iterate through images in the directory
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_directory, filename)

        # Process the image
        output, mask, num_items = process_image(image_path)
        if output is None:
            continue

        # Save the output image and mask
        output_path = os.path.join(output_directory, 'output_' + filename)
        mask_path = os.path.join(output_directory, 'mask_' + filename)
        cv2.imwrite(output_path, output)
        cv2.imwrite(mask_path, mask)

        # Print the number of items
        print(f'Number of items in {filename}: {num_items}')


Number of contours detected: 50
Number of filtered contours: 49
Number of items in img1.jpg: 49
Number of contours detected: 48
Number of filtered contours: 45
Number of items in img4.jpg: 45
Number of contours detected: 46
Number of filtered contours: 44
Number of items in img3.jpg: 44
Number of contours detected: 49
Number of filtered contours: 48
Number of items in img1_43_nosy.jpg: 48
Number of contours detected: 51
Number of filtered contours: 49
Number of items in img6.jpg: 49
Number of contours detected: 48
Number of filtered contours: 46
Number of items in img2.jpg: 46
Number of contours detected: 50
Number of filtered contours: 47
Number of items in img5.jpg: 47


In [ ]:
import cv2
import numpy as np
import os

# Path to your image directory and output directory
image_directory = '/content/drive/MyDrive/Screws_2024_07_15'
output_directory = '/content/drive/MyDrive/CountingChallenge/Non_AI/output'
os.makedirs(output_directory, exist_ok=True)

def process_image(image_path):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f'Failed to load image: {image_path}')
        return None, 0

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Image preprocessing
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)

    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Morphological operations
    kernel = np.ones((3, 3), np.uint8)
    morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    morphed = cv2.morphologyEx(morphed, cv2.MORPH_OPEN, kernel, iterations=1)

    # Find contours
    contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area
    min_contour_area = 100
    max_contour_area = 5000
    filtered_contours = [c for c in contours if min_contour_area < cv2.contourArea(c) < max_contour_area]

    # Draw bounding boxes around the detected screws
    for contour in filtered_contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Count the number of contours
    num_items = len(filtered_contours)

    return image, num_items

# Iterate through images in the directory
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_directory, filename)

        # Process the image
        output_image, num_items = process_image(image_path)
        if output_image is None:
            continue

        # Save the output image with bounding boxes
        output_path = os.path.join(output_directory, 'output_' + filename)
        cv2.imwrite(output_path, output_image)

        # Print the number of items detected
        print(f'Number of items in {filename}: {num_items}')


Number of items in img1.jpg: 49
Number of items in img4.jpg: 45
Number of items in img3.jpg: 44
Number of items in img1_43_nosy.jpg: 48
Number of items in img6.jpg: 49
Number of items in img2.jpg: 46
Number of items in img5.jpg: 47
